In [4]:
!pip install qiskit

In [8]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator

In [9]:
import numpy as np
from math import pi, sqrt
import random

In [10]:
# the basis gates
basis_g = ['id', 'rz', 'sx', 'x', 'cx']

#creating the gate statistics list
gate_stats = []

#number of qbits being controlled
ctrl_qbits = 0

In [11]:
# single qubit gates with no parameters
sgle_noparam = ["i","x","y","z","h","s","sdg","t","tdg","sx","sxdg"]

for gate in sgle_noparam:
    circuit = QuantumCircuit(1)
    getattr(circuit, gate)(0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    gate_stats.append([gate, ctrl_qbits, transpile_circuit.size()])

In [12]:
# single qubit gates with 1 parameter
sgle_1_param = ["p","rx","ry","rz"]

for gate in sgle_1_param:
    param = random.uniform(-np.pi, np.pi)
    circuit = QuantumCircuit(1)
    getattr(circuit, gate)(param,0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    gate_stats.append([gate,ctrl_qbits, transpile_circuit.size()])

In [13]:
# single qubit gates with 2 parameters
single_2_param = ["u2"]

for gate in single_2_param:
    circuit = QuantumCircuit(1)
    rand_angle1 = random.uniform(-np.pi, np.pi)
    rand_angle2 = random.uniform(-np.pi, np.pi)
    getattr(circuit, gate)(rand_angle1, rand_angle2, 0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    gate_stats.append([gate,ctrl_qbits, transpile_circuit.size()])

/var/folders/7l/z060s3ts4sb0kyt487t13pdw0000gn/T/ipykernel_13597/3578139103.py:8: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(λ-π/2) (1 pulse on hardware).
  getattr(circuit, gate)(rand_angle1, rand_angle2, 0)


In [14]:
# single qubit gates with 3 parameters
single_3_param = ["u", "u3"]

for gate in single_3_param:
    circuit = QuantumCircuit(1)
    rand_angle1 = random.uniform(-pi, pi)
    rand_angle2 = random.uniform(-pi, pi)
    rand_angle3 = random.uniform(-pi, pi)
    getattr(circuit, gate)(rand_angle1, rand_angle2, rand_angle3, 0)
    transpile_circuit = transpile(circuit, basis_gates = basis_g)
    gate_stats.append([gate,ctrl_qbits, transpile_circuit.size()])

/var/folders/7l/z060s3ts4sb0kyt487t13pdw0000gn/T/ipykernel_13597/2597784990.py:9: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  getattr(circuit, gate)(rand_angle1, rand_angle2, rand_angle3, 0)


In [15]:
gate_stats

[['i', 0, 0],
 ['x', 0, 1],
 ['y', 0, 2],
 ['z', 0, 1],
 ['h', 0, 3],
 ['s', 0, 1],
 ['sdg', 0, 1],
 ['t', 0, 1],
 ['tdg', 0, 1],
 ['sx', 0, 1],
 ['sxdg', 0, 3],
 ['p', 0, 1],
 ['rx', 0, 5],
 ['ry', 0, 4],
 ['rz', 0, 1],
 ['u2', 0, 3],
 ['u', 0, 5],
 ['u3', 0, 5]]

In [16]:
# creating a text file to store the gate stats(profiles)
import os
cd = os.getcwd()
path = cd + "/gate_stats.txt"
f = open(path, 'w')

In [17]:
# writing the profiles to text file
f = open("gate_stats.txt", "a")
for i in np.arange(len(gate_stats)):
    f.writelines([gate_stats[i][0], " ", str(gate_stats[i][1]), " ", str(gate_stats[i][2])])
    f.writelines(['\n'])
f.close()